# The mock object

In [5]:
from unittest.mock import Mock

mock = Mock()
mock

<Mock id='140334170933736'>

## Lazy attributes and methods

In [6]:
print(mock.do_something())
print(mock.some_attribute)

<Mock name='mock.do_something()' id='140334170935024'>
<Mock name='mock.some_attribute' id='140334170934912'>


## MagicMock: also creates magic methods

In [7]:
print(len(mock))

TypeError: object of type 'Mock' has no len()

In [8]:
from unittest.mock import MagicMock

magic_mock = MagicMock()

print("len(magic_mock):", len(magic_mock))
print("int(magic_mock):", int(magic_mock))

len(magic_mock): 0
int(magic_mock): 1


## Speccing
### Specify object interface

In [9]:
dog_mock = Mock(spec=["bark", "sit"])
dog_mock.bark()

<Mock name='mock.bark()' id='140334170988728'>

In [10]:
dog_mock.meow()

AttributeError: Mock object has no attribute 'meow'

### Autospeccing: take from existing object

In [11]:
class Cat:
    def meow(self):
        return "Meow, meow!"

tibbles = Cat()
tibbles.meow()

'Meow, meow!'

In [12]:
cat_mock = Mock(spec=tibbles)
cat_mock.meow()

<Mock name='mock.meow()' id='140334170991472'>

In [13]:
cat_mock.bark()

AttributeError: Mock object has no attribute 'bark'

### create_autospec
* shorthand for Mock(spec=...)
* can be specified in patch -> we'll see later

In [14]:
from unittest.mock import create_autospec

auto_cat_mock = create_autospec(tibbles)
auto_cat_mock.meow()

<MagicMock name='mock.meow()' id='140334170992480'>

In [15]:
auto_cat_mock.bark()

AttributeError: Mock object has no attribute 'bark'

# Assertions and inspections

In [16]:
horse_mock = Mock()

horse_mock.gallop()

horse_mock.eat("carrot")
horse_mock.eat("apple", speed="quick")

<Mock name='mock.eat()' id='140334170723664'>

## Assertions

In [17]:
horse_mock.eat.assert_called()
horse_mock.eat.assert_called_with("apple", speed="quick")

horse_mock.gallop.assert_called_once()

horse_mock.drink.assert_not_called()

In [18]:
horse_mock.drink.assert_called()

AssertionError: Expected 'drink' to have been called.

In [19]:
horse_mock.eat.assert_called_with("apple")

AssertionError: Expected call: eat('apple')
Actual call: eat('apple', speed='quick')

## Inspecting calls to the mock

In [20]:
print("call_count:    ", horse_mock.eat.call_count)

print("call_args:     ", horse_mock.eat.call_args)

print("call_args_list:", horse_mock.eat.call_args_list) 

print("method_calls:  ", horse_mock.method_calls)

call_count:     2
call_args:      call('apple', speed='quick')
call_args_list: [call('carrot'), call('apple', speed='quick')]
method_calls:   [call.gallop(), call.eat('carrot'), call.eat('apple', speed='quick')]


# Configuring behaviour

## Return value

In [21]:
horse_mock.eat.return_value = "Mmh, this was yummy!"

horse_mock.eat("apple")

'Mmh, this was yummy!'

## Side effect

### List of return values

In [25]:
horse_mock.eat.side_effect = [
    "Mmh, this was yummy!",
    "I am getting full...",
    "Stop feeding me!"
]

print(horse_mock.eat("apple"))
print(horse_mock.eat("apple"))
print(horse_mock.eat("apple"))

Mmh, this was yummy!
I am getting full...
Stop feeding me!


In [26]:
print(horse_mock.eat("apple"))

StopIteration: 

### Function

In [32]:
def mock_eat(food):
    return "This {} was yummy!".format(food)

horse_mock.eat = Mock(side_effect = mock_eat) # Mocks can be configured in the constructor

print(horse_mock.eat("apple"))
print(horse_mock.eat("pear"))

This apple was yummy!
This pear was yummy!


### Exception

In [30]:
horse_mock.eat.side_effect = Exception("I am not hungry now.")

horse_mock.eat("apple")

Exception: I am not hungry now.

# Let's write actual tests!